# Análisis de datos

In [3]:
from functools import reduce
from textwrap import wrap
import re
import math

In [4]:
with open("borrador_nueva_constitución.txt", 'r') as file:
    text = file.read()
    arts = re.split("-"*50, text)
print(len(arts))

499


Largo del artículo más largo:

In [21]:
max_art = max(arts, key=len)
print("Artículo:", max_art.split(".-")[0].strip())
print("Largo:", len(max_art))
print("Tweets aproximados:", math.ceil(len(max_art) / 269))

Artículo: 441
Largo: 4579
Tweets aproximados: 17


Artículo con más incisos:

In [23]:
arts_splited = (art.strip().split('\n') for art in arts)
max_art = max(arts_splited, key=len)
print("Artículo:", max_art[0].split(".-")[0].strip())
print("Incisos:", len(max_art))

Artículo: 186
Incisos: 27


## Generación de tweets

In [30]:
def get_tweets(art, max_len=280):
    '''
    Dado un artículo (srt), retorna una lista de tweets con el artículo.
    '''
    # Se eliminan los espacios en blanco en los extremos
    art = art.strip()
    # Si el artículo es muy corto, se devuelve un solo tweet
    if len(art) <= max_len:
        return [art]
    # En caso contrario, se calculan los tweets necesarios
    tweets = []
    # Se resta de max_len es espacio necesario para codificar el índice de cada tweet
    max_len -= len(" (XX/XX)")
    # Se separa el artículo en incisos
    clauses = art.split('\n')
    # Se recorren los incisos
    i = 0
    while i < len(clauses):
        actual_tweet = clauses[i]
        # Si el inciso es corto, se intenta añadir otro inciso en el mismo tweet
        if len(actual_tweet) <= max_len:
            actual_tweet_new = actual_tweet
            while len(actual_tweet_new) < max_len and i < len(clauses):
                actual_tweet = actual_tweet_new
                i += 1
                if i < len(clauses):
                    actual_tweet_new += '\n' + clauses[i]
            tweets.append(actual_tweet)
        # Si el inciso es muy largo, se divide en varios tweets
        else:
            incise_tweets = get_incise_tweets(actual_tweet, max_len - 2*len("..."))
            tweets.extend(incise_tweets)
        i += 1
    # Añadimos el índice de cada tweet
    tweets = list(tweet + f" ({i}/{len(tweets)})" for i, tweet in enumerate(tweets, start=1))
    return tweets

def get_incise_tweets(incise, max_len):
    '''
    Dado un inciso, retorna una lista de tweets con el inciso.
    '''
    # Se eliminan los espacios en blanco en los extremos
    incise = incise.strip()
    # Se divide el inciso en bloques del largo de un tweet
    tweets = wrap(incise, max_len)
    # Se recorren los tweets generados
    for i in range(len(tweets)):
        # Si el tweet no es inicial y el tweet anterior no termina con un punto, se añaden puntos suspensivos al inicio
        if i > 0:
            if not re.search("[^.][.]", tweets[i-1][-2:]):
                tweets[i] = "..." + tweets[i]
        # Si el tweet no es final y no termina con un punto, se añaden puntos suspensivos al final
        if i < len(tweets) - 1:
            if not re.search("[^.][.]", tweets[i][-2:]):
                tweets[i] = tweets[i] + "..."
    return tweets

def n_tweets(art):
    '''
    Dado un artículo, retorna el número de tweets necesarios para publicarlo.
    '''
    return len(get_tweets(art))

Artículo que requiere más tweets:

In [33]:
max_art = max(arts, key=n_tweets)
print("Artículo:", max_art.split(".-")[0].strip())
print("Tweets:", n_tweets(max_art))

Artículo: 441
Tweets: 15


Tweet más largo de todos:

In [48]:
max_len = 0
tweet = ""
for art in arts:
    act_twt = max(get_tweets(art), key=len)
    act_len = len(act_twt)
    if act_len > max_len:
        max_len = act_len
        tweet = act_twt
print("Largo:", max_len)
print("Tweet:\n", tweet)

Largo: 280
Tweet:
 20.- Artículo 16.- Las diputadas y diputados y las y los representantes regionales podrán ser reelegidos sucesivamente en el cargo hasta por un período. Para estos efectos se entenderá que han ejercido su cargo durante un período cuando han cumplido más de la mitad de su mandato.
